In [1]:
#1. rutin1 import module
import pandas as pd
import os
import sys

In [2]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [3]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "notebooks/postager_nlp-id/dataset_ekstraksi_r29_pos_sm.xlsx")
df = pd.read_excel(dataset_path)
df_pos = df['pos_sentence_list']

In [4]:
from multipartiterank import MultipartiteRank

pos = {'NN', 'VP', 'NP'}
# 1. create a TextRank extractor.
extractor = MultipartiteRank()

In [5]:
import ast 

df_keyphrases = pd.DataFrame(columns=['key_1', 'val_1', 'key_2', 'val_2', 'key_3', 'val_3'])

for i in range(len(df_pos)):
    try:
        text = df_pos[i]
        print('Processing Text', i, end='..! ')
        text1 = ast.literal_eval(text)
        extractor.load_document(input=text1)
        keyphrase = extractor.candidate_weighting()

        # Separate the key and value elements into separate lists
        keys = [item[0] for item in keyphrase]
        values = [item[1] for item in keyphrase]

        # Create a dictionary with the keys and values
        data = {
            'key_1': [keys[0]],
            'val_1': [values[0]],
            'key_2': [keys[1]],
            'val_2': [values[1]],
            'key_3': [keys[2]],
            'val_3': [values[2]],
        }
        df_keyphrase = pd.DataFrame(data)
        print('Done')
    except Exception as e:
        print('Error on text', i)
        data = {
            'key_1': ['na'],
            'val_1': [0],
            'key_2': ['na'],
            'val_2': [0],
            'key_3': ['na'],
            'val_3': [0],
        }
        df_keyphrase = pd.DataFrame(data)
    # Concatenate df_keyphrase with df_keyphrases
    df_keyphrases = pd.concat([df_keyphrases, df_keyphrase], ignore_index=True)


Processing Text 0..! Error on text 0
Processing Text 1..! Error on text 1
Processing Text 2..! Error on text 2
Processing Text 3..! Error on text 3
Processing Text 4..! Error on text 4
Processing Text 5..! Error on text 5
Processing Text 6..! Error on text 6
Processing Text 7..! Error on text 7
Processing Text 8..! Error on text 8
Processing Text 9..! Error on text 9
Processing Text 10..! Error on text 10
Processing Text 11..! Error on text 11
Processing Text 12..! Error on text 12
Processing Text 13..! Error on text 13
Processing Text 14..! Error on text 14
Processing Text 15..! Error on text 15
Processing Text 16..! Error on text 16
Processing Text 17..! Error on text 17
Processing Text 18..! Error on text 18
Processing Text 19..! Error on text 19
Processing Text 20..! Error on text 20
Processing Text 21..! Error on text 21
Processing Text 22..! Error on text 22
Processing Text 23..! Error on text 23
Processing Text 24..! Error on text 24
Processing Text 25..! Error on text 25
Proces

In [6]:
# Add df_keyphrases ke file excel dataset_ekstraksi_r30.xlsx
result_df = pd.concat([df, df_keyphrases], axis=1)

# simpan dalam dataset
#output_file = 'std_pke_textrank.xlsx'
#result_df.to_excel(output_file, index=False)

EVALUATION

In [7]:
from utils import eval, write_excel

targets = result_df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [8]:
# Evaluation TextRank
predict_textrank_list = result_df[['key_1','key_2','key_3']].values.tolist()
eval_textrank = eval(predict_textrank_list, targets, True).round(3)
eval_textrank.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank = eval_textrank[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_textrank.head(3)

,key_1,key_2,key_3,flex_recall,flex_prec
0,partial_match,partial_match,partial_match,0.429,1.0
1,partial_match,partial_match,partial_match,0.429,1.0
2,partial_match,partial_match,partial_match,0.429,1.0


In [9]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall = eval_textrank['flex_recall'].mean()
textrank_prec = eval_textrank['flex_prec'].mean()
textrank_f1 = 2 * (textrank_prec * textrank_recall) / (textrank_prec + textrank_recall)

# Create a DataFrame with the scores
summary = pd.DataFrame({'textrank': [textrank_recall, textrank_prec, textrank_f1]}, index=['recall', 'precision', 'F1'])
summary = summary.round(3)
summary

,textrank
recall,0.429
precision,1.000
F1,0.600


In [10]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank = pd.concat([result_df, df_targets, eval_textrank], axis=1)
predict_textrank.head(3)

,No,Letter No,Letter No_,tanggal,judul,isi,pos_filter,Annotator,k1,k2,...,2,3,4,5,6,key_1,key_2,key_3,flex_recall,flex_prec
0,1,001/CPQ101/2017-S0,001_CPQ101_2017-S0,2017-10-30,Usulan Personil Penting\n(Proposed key personnel),Menunjuk surat KONTRAKTOR No. JTB-L-RJ-PJ-B000...,usulan personil surat kontraktor tertanggal us...,A,persetujuan tertulis,prosedur,...,usulan,pengganti,NaN,NaN,NaN,partial_match,partial_match,partial_match,0.429,1.0
1,2,002/CPQ101/GPF/2017-S0,002_CPQ101_GPF_2017-S0,2017-11-06,Template Document JTB - GPF Project,Mengacu kepada dokumen : JTB-CP-B-CTR-900-0000...,dokumen sampaikan proyek gas disampaikan acuan...,A,template document,exhibit c,...,acuan,pengelolaan,dokumen,NaN,NaN,partial_match,partial_match,partial_match,0.429,1.0
2,3,003/CPQ101/GPF/2017-S0,003_CPQ101_GPF_2017-S0,2017-11-06,Change Inquiry Terkait Usulan Perubahan Lingku...,Mengacu pada risalah rapat mingguan proyek JTB...,terkait usulan perubahan lingkup kerja terkait...,A,ruang kantor,change inquiry,...,lingkup kerja,akomodasi,services for company,exhibit a,NaN,partial_match,partial_match,partial_match,0.429,1.0


In [11]:
# Write predictions to excel file
sheet_name = 'pke_multipartiterank'
output_file = 'pke_multipartiterank.xlsx'
write_excel(predict_textrank, sheet_name, output_file)

/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:16: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:25: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
